<a href="https://colab.research.google.com/github/vikion/slovak-news-summarization/blob/master/mbart1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached datasets-2.10.1-py3-none-any.whl (469 kB)
  Using cached multiprocess-0.70.14-py39-none-any.whl (132 kB)
  Using cached xxhash-3.2.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached aiohttp-3.8.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached multidict-6.0.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# Preparing dataset

In [31]:
import zipfile
import pandas as pd
from ast import literal_eval

pluska = []
 
# open zipped dataset
with zipfile.ZipFile("pluska2.zip", 'r') as z:

   for filename in z.namelist():
      if filename != "pluska2/" and '__MACOSX' not in filename:
        with z.open(filename) as f:
            # read the dataset
            info = f.read()
            my_dict = literal_eval(info.decode('utf-8'))

            pluska.append(my_dict)
            
            # display dataset
print(pluska[0])
df_pluska = pd.DataFrame(pluska)

{'title': 'FOTO Diváci Tvojej tváre sa poriadne obuli do herečky Siposovej: Vyzeráš ako japonský transvestita!', 'sum': 'Markizácka šou Tvoja tvár znie povedome priniesla aj vo svojom 3 . kole rôzne momenty. Všetkých osem súťažiacich sa premenilo na svetové spevácke hviezdy a predviedli skvelé vystúpenia. Jedným takým bol aj spev Lucie Siposovej, ktorá prekvapila mnohých divákov. No jej premena niektorých naopak poriadne zaskočila.', 'text': 'Tretie kolo Tvojej tváre prinieslo, ako sa hovorí, z\xa0každého rožka niečo. Videli sme rock, pop, R&B i\xa0slovenskú klasiku, ktorú bravúrne predviedol  herec René Štúr , ktorý sa v\xa0tento večer premenil na\xa0slovenskú spevácku legendu Michala Dočolomanského. Príjemne prekvapila aj  herečka Lucia Siposová,  ktorá zaspievala  legendárny hit "Stop" od\xa0britskej speváčky Sam Brown . Jej premena však  šokovala viacerých divákov , pričom niektorí ju prirovnali  k japonskému transvestitovi . "Ty si jedna prefíkaná mačka, máš úžasnú farbičku v\xa0h

In [32]:
import datasets
import pandas as pd
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
from sklearn.model_selection import train_test_split


x_train ,x_test = train_test_split(pluska,test_size=0.8)

x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)

train_dataset = Dataset.from_pandas(x_train)
test_dataset = Dataset.from_pandas(x_test)
my_dataset_dict = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})

[{'title': 'Bartoška podcenil svoju vážnu chorobu: Čo mu priťažilo počas nakrúcania Tigra?', 'sum': 'Keď ikone československého filmu Jiřímu Bartoškovi pred dvomi rokmi diagnostikovali rakovinu lymfatických uzlín, fanúšikovia mu držali palce. Boj s chorobou charizmatickému hercovi neubral ani štipku z jeho povestného optimizmu a šarmu. Týždenníku Šarmu však priznal, že sa podpísala pod jeho fyzické výkony v práci.', 'text': 'Minulý rok, len krátko po skončení chemoterapií, sa ako hlavný hrdina filmu Teória tigra plný elánu pustil do\xa0nakrúcania. No rýchlo zistil, že telo nemá ešte v\xa0takej kondícii, ako dúfal. Situáciu mu neuľahčovalo ani počasie, keďže film vznikal v\xa0štyridsaťstupňových teplotách.  „Keď sme začali točiť, už som nebral žiadnu liečbu. Ešte predtým som stihol urobiť aj päťdesiaty ročník festivalu v\xa0Karlových Varoch. Čiže mal som už nejaký čas po chemoterapii aj po ožarovaniach, ale telo sa predsa len čistí a\xa0regeneruje trošku dlhšie, než som si myslel. Kým j

In [33]:
##print samples from dataset
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['title']}'")
        print(f"'>> Review: {example['text']}'")

# Tokenizing dataset



In [34]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [35]:
max_input_length = 512
max_target_length = 30

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [36]:
tokenized_datasets = my_dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

# Rouge

In [44]:
!pip install rouge_score
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=3bf75c24370c22253419daaba51cca3228a02e6847c1ef239e523442f4efcfd0
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.3 MB/s eta 0:00:00


In [45]:
import evaluate

rouge_score = evaluate.load("rouge")


# Optimalization?

In [38]:
pip install transformers datasets accelerate nvidia-ml-py3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19190 sha256=2674140fed044b63a20c45c6a20ef7fa7b02405b2d9f29a4678b5bcc97537b3e
  Stored in directory: /root/.cache/pip/wheels/f6/d8/b0/15cfd7805d39250ac29318105f09b1750683387630d68423e1
Successfully built nvidia-ml-py3


In [39]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [61]:
print_gpu_utilization()

GPU memory occupied: 14393 MB.


In [41]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()
torch.cuda.empty_cache()

In [60]:
!nvidia-smi

Sat Mar 11 10:35:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    33W /  70W |  14135MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Preparing model (?)

In [48]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_name.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-pluska_sum",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True
)


In [49]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [50]:
print(my_dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['title', 'sum', 'text', 'url'],
        num_rows: 250
    })
    test: Dataset({
        features: ['title', 'sum', 'text', 'url'],
        num_rows: 250
    })
})


In [51]:
tokenized_datasets = tokenized_datasets.remove_columns(
    my_dataset_dict["train"].column_names
)

In [52]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[250004,  24169,  17988,  ...,  12373,   7144,      2],
        [250004,  31485, 132710,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[250004,   3253,    188,   7659,    855,  13844,    379,  15060, 107804,
             34,  98838,     34,     12,  28000,    842,    494,  50141,   2821,
            365,  25866,   2593,  11780,    408,   1174,   2371,   5739,     32,
              2],
        [250004, 218528,  41337,     42,  86805,     32,   2291,   3104,    430,
             57,     36, 193768,     13, 209723,    515,  13562,      2,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100]]), 'decoder_input_ids': tensor([[     2, 250004,   3253,    188,   7659,    855,  13844,    379,  15060,
         107804,     34,  98838,     34,     12,  28000,    842,    494,  50141,
           2821,    365,  25866,   2593,  11780,   

In [56]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Fine-tuning

In [57]:
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [58]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/kiviki/mbart-large-50-pluska_sum into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/2.28G [00:00<?, ?B/s]

Download file runs/Mar10_15-47-27_3319b76a4d78/events.out.tfevents.1678463755.3319b76a4d78.184.4: 100%|#######…

Clean file runs/Mar10_15-47-27_3319b76a4d78/events.out.tfevents.1678463755.3319b76a4d78.184.4:  12%|#1        …

Download file runs/Mar10_15-47-27_3319b76a4d78/1678463755.8446255/events.out.tfevents.1678463755.3319b76a4d78.…

...

Clean file runs/Mar10_15-45-13_3319b76a4d78/events.out.tfevents.1678463138.3319b76a4d78.184.0:  22%|##1       …

Clean file training_args.bin:  28%|##8       | 1.00k/3.56k [00:00<?, ?B/s]

Clean file sentencepiece.bpe.model:   0%|          | 1.00k/4.83M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/16.3M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/2.28G [00:00<?, ?B/s]

In [59]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 250
  Num Epochs = 8
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 4
  Total optimization steps = 168
  Number of trainable parameters = 610879488


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,6.902801,14.251600,4.171800,12.412000,12.663500
2,No log,5.673759,14.741800,4.370800,12.761600,13.101800
3,No log,5.548406,15.000400,3.521700,12.631000,12.905300


***** Running Evaluation *****
  Num examples = 250
  Batch size = 3
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_tok

OutOfMemoryError: ignored

In [ ]:
trainer.evaluate()

In [ ]:
client.close()